# Installation

Run the folowing 3 cells to install the required packages as well as import then and then hide the warnings.

In [5]:
#!pip install -r requirements.txt

In [2]:
import maskpass  # to hide/mask the password
import psycopg2
import math
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import polygonize
import fiona
import matplotlib.pyplot as plt
import shapely
import numpy as np
import csv
import pandas as pd
from shapely import wkt
import folium
from gcloud import storage
import getpass
import warnings
import subprocess

In [3]:
warnings.filterwarnings('ignore')

# Functions

Run the cells below to have all the required functions.

In [7]:
def db_conn():
    # Masking the password, the user will be asked for the password
    #pwd = maskpass.askpass(mask="") 
    # To connect to database. Update IP, port, database and user to your values.
    conn = psycopg2.connect(
        host="34.159.36.105",#"localhost",
        port ="5432",
        database="geodp",
        user="postgres", 
        password='postgres')#pwd)
    cur = conn.cursor()

    return cur, conn

In [9]:
def get_geometry(table):
    cursor, conn = db_conn()
    cursor.execute("SELECT st_x(geom), st_y(geom), state FROM "+table+";")
    rows = cursor.fetchall()
    cursor.close()

    result = []
    for index in range(len(rows)):
        result.append([
            rows[index][0],
            rows[index][1],
            rows[index][2]
            
        ])
    
    return result

In [30]:
def get_initial_points(table):
    l = get_geometry(table)
    x = []
    y = []
    states = []
    for i in l:
        x.append(i[0])
        y.append(i[1])
        states.append(i[2])
    states = list(set(states))
    points = gpd.GeoDataFrame({"x":x,"y":y})
    points['geometry'] = points.apply(lambda p: Point(p.x, p.y), axis=1)
    points_unique = points.drop_duplicates()

    return points_unique,states

In [8]:
def get_grid_size(n,eps):
    c = 10
    #epsilon = 0.1
    #print(n)
    m = math.sqrt((n*eps)/c)
    #print(m)
    return m

In [15]:
#initial_points,states_list = get_initial_points("smalldata")

In [11]:
def create_grid(points_unique,n,eps):
    cell_size = get_grid_size(n,eps) #0.7616 
    xmin, ymin, xmax, ymax= points_unique.total_bounds
    crs = "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs"
    # create the cells 
    grid = []
    for x0 in np.arange(xmin, xmax+cell_size, cell_size):
        for y0 in np.arange(ymin, ymax+cell_size, cell_size):
            x1 = x0-cell_size
            y1 = y0+cell_size
            grid.append(shapely.geometry.box(x0, y0, x1, y1))
    cell = gpd.GeoDataFrame(grid, columns=['geometry'], 
                                    crs=crs)

    return cell

In [17]:
#cell = create_grid(initial_points, len(initial_points))

In [41]:
def plot_points(points,cells,states_list):
    
    states = gpd.read_file('states/tl_2021_us_state.shp')
    states = states.to_crs("EPSG:4326")
    states.boundary.plot(color = 'grey')
    #print(states[states['NAME'].isin(states_list)])
    base = states[states['NAME'].isin(states_list)].plot(color='white', edgecolor='black',figsize=(12, 8))
    plt.autoscale(False)
    points.plot(ax=base, marker='o', color='blue', markersize=5)
    cells.plot(ax=base, facecolor="none", edgecolor='grey')
    base.axis("off")

In [12]:
#plot_points(initial_points,cell,states_list)

In [13]:
def get_cell_counts(points_unique,cell, eps):
    cell = cell.reset_index().rename(columns = {'index':'id'})
    #join the cells and points and remove duplicates if there are
    pointInPolygons = gpd.sjoin(points_unique, cell, how='inner', op = 'intersects')
    pointInPolygons = pointInPolygons.drop_duplicates(subset=['x', 'y'], keep='first')
    #count the initial points
    count_points = pointInPolygons.groupby(['id']).size().reset_index(name='count')
    cell_counts = pd.merge(cell,count_points, on = 'id')
    
    #apply the noise and round the counts
    #eps = 0.1
    dp_count = []
    for row in cell_counts.iterrows():
        dp_count.append(row[1]['count']+np.random.laplace(0, 1 / eps, 1)[0])
    cell_counts['count_dp'] = dp_count
    cell_counts['count_dp'] = cell_counts['count_dp'].apply(lambda x : x if x > 0 else 0)
    cell_counts['count_dp_rounded'] = round(cell_counts['count_dp'])
    #cell_counts[cell_counts['count_dp_rounded']<0]
    return cell_counts


In [24]:
#cell_counts = get_cell_counts(initial_points, cell)

In [14]:
def get_polygons(cell_counts):
    polygons = {}
    for i in range(0,cell_counts.count()[0]):
        polygons[wkt.dumps(cell_counts['geometry'][i])] = cell_counts['count_dp_rounded'][i]
    return polygons

In [15]:
def new_points(polygon,n):
    cursor, conn = db_conn()
    cursor.execute("select st_asText(ST_GeneratePoints(ST_GeomFromText('"+polygon+"'),"+str(n)+"));")
    rows = cursor.fetchall()
    cursor.close()

    result = []
    for index in range(len(rows)):
        result.append([
            rows[index][0]
            
        ])
    points = result[0][0].replace('MULTIPOINT(','').replace(")","")
    point_list = points.split(",")
    x_y = []
    for i in point_list:
        x_y.append(i.split(" "))
    return x_y

In [16]:
def get_all_new_points(polygons):
    all_new_points = []
    for pol in polygons:
        if polygons[pol].astype('int')!=0:
        
            p = new_points(pol,polygons[pol].astype('int'))
            all_new_points.append(p)
    
    return all_new_points

In [30]:
'''cell_counts = get_cell_counts(initial_points,cell)
polygons = get_polygons(cell_counts)
all_new_points = get_all_new_points(polygons)'''

In [17]:
def create_new_table():
    cursor, conn = db_conn()
    cursor.execute("CREATE TABLE IF NOT EXISTS  test  ( new_id int4 primary key, new_geom geometry(POINT,4326) );")
    conn.commit()
    cursor.close()

In [18]:
def generate_df_for_new_table(points):
    point_dict = {'before':[],'x': [],'between':[], 'y': [], 'after':[]}
    for pol in points:
        
        for point in pol:
            
            point_dict['before'].append('SRID=4326;POINT(')
            point_dict['x'].append(point[0])
            point_dict['between'].append(' ')
            point_dict['y'].append(point[1])
            point_dict['after'].append(')')
            
    points_df = pd.DataFrame.from_dict(point_dict)
    return points_df

In [19]:
def generate_csv_for_new_table(df):
    df = df.reset_index().rename(columns = {'index':'id'})
    df["geom"] = df[["before","x", "between","y", "after"]].apply("".join, axis=1)
    df = df.drop(columns=["before","x", "between","y", "after"])
    
    df.to_csv('points.csv',index = False)
    #put the file to the vm
    rc = subprocess.call("./upload_csv.sh", shell=True)

In [104]:
'''def generate_new_points_df(all_new_points):
    df_new_points = generate_df_for_new_table(all_new_points).drop(columns=["before", "between", "after"])
    new_points_gpd = gpd.GeoDataFrame(
    df_new_points, geometry=gpd.points_from_xy(df_new_points['x'], df_new_points['y']))
    return new_points_gpd'''

In [105]:
#df = generate_df_for_new_table(all_new_points)
#generate_csv_for_new_table(df)
#newpoints = generate_new_points_df(all_new_points)

In [20]:
def insert_csv_into_new_table():
    cursor, conn = db_conn()
    cursor.execute("delete from test;")
    user = getpass.getuser()
    cursor.execute("COPY test FROM '/home/ingastrelnikova28_gmail_com/points.csv' DELIMITERS ',' CSV HEADER;")
    conn.commit()
    cursor.close()

In [21]:
def bounding_box():
    cursor, conn = db_conn()
    cursor.execute("select ST_AsText(ST_Envelope(ST_Collect(geom))) as bounding_box from test;")
    rows = cursor.fetchall()
    result = []
    for index in range(len(rows)):
        result.append([
            rows[index][0]
        ])
    cursor.close()
    polygon_postgis = result[0]
    x_y = []
    x = []
    y = []
    polygon_postgis = polygon_postgis[0].replace('POLYGON((','').replace("))","")
    points_list = polygon_postgis.split(",")
    for i in points_list:
        x.append(i.split(" ")[0])
        y.append(i.split(" ")[1])
    lon_point_list = list(map(float, x))
    lat_point_list = list(map(float, y))
    polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom]) 
    coord_lon = int(lon_point_list[-1])
    coord_lat = int(lat_point_list[-1])
    
    m = folium.Map([coord_lat, coord_lon], zoom_start=4, tiles='cartodbpositron')
    folium.GeoJson(polygon).add_to(m)
    folium.LatLngPopup().add_to(m)
    polygon_val = polygon._get_value(0,'geometry')
    return m, polygon_val

In [22]:
def geom_center():
    cursor, conn = db_conn()
    cursor.execute("SELECT avg(ST_X(geom)) as lon, avg(ST_Y(geom)) as lat FROM test;")
    result = cursor.fetchone() 
    m = folium.Map([result[1], result[0]], zoom_start=4, tiles='cartodbpositron')
    folium.CircleMarker(location=[result[1], result[0]],
                        radius=2,
                        weight=5).add_to(m)
    return m, result

In [41]:
def client_request(query,eps):

    if ("select" not in query) or ("from" not in query) or (";" not in query):
        print("Please use a correct SQL query!")
        return

    table = query.partition("from")[2]
    
    #In case the user wants to filter the data for date or counte/state, create a new table with filtered data
    if "where" in query:
        query_list = query.partition("where")
        conditions = query_list[2]
        table = query_list[0].partition("from")[2]
        
        new_query = "CREATE TABLE filtered_data AS SELECT * from "+ table+" where "+conditions

        cursor, conn = db_conn()
        cursor.execute("DROP TABLE filtered_data;")
        cursor.execute(new_query)
        conn.commit()
        cursor.close()

    #eps = 0.1

    #Apply differencial privacy
    
    initial_points,states_list = get_initial_points(table)
    grid_cells = create_grid(initial_points,len(initial_points),eps)
    cell_counts = get_cell_counts(initial_points,grid_cells,eps)

    polygons = get_polygons(cell_counts)
    new_points = get_all_new_points(polygons)

    create_new_table()
    points_df = generate_df_for_new_table(new_points)
    generate_csv_for_new_table(points_df)
    insert_csv_into_new_table()

    #dp.plot_points(initial_points,grid_cells,states_list)
    #dp.plot_points(new_points,grid_cells,states_list)
    
    
    #Check what function
    if "st_envelope" in query.lower():
        bounding_box_map, polygon = bounding_box()
        print(polygon)
        return bounding_box_map, polygon
    elif "st_centroid" in query.lower():
        point_map, center=geom_center()
        print(center)
        return point_map, center

# Presentation

Extracting the table from the user request

In [38]:
bb, bb_val = client_request("select ST_AsText(ST_Envelope(ST_Collect(geom))) from mediumdata where state = 'California' or state = 'Texas';",0.1)

Using OS Login user [ingastrelnikova28_gmail_com] instead of requested user [ingastrelnikova28]


POLYGON ((-161.5286990901236 20.629662694022283, -161.5286990901236 69.65517410804746, -66.49990065626243 69.65517410804746, -66.49990065626243 20.629662694022283, -161.5286990901236 20.629662694022283))


(<function __main__.bounding_box()>,
 <shapely.geometry.polygon.Polygon at 0x7f95b1b909d0>)

In [39]:
bb, bb_val = client_request("select ST_AsText(ST_Envelope(ST_Collect(geom))) from mediumdata where state = 'California' or state = 'Texas';",0.1)

Using OS Login user [ingastrelnikova28_gmail_com] instead of requested user [ingastrelnikova28]


POLYGON ((-166.41728126401668 19.645674168881076, -166.41728126401668 69.47489185423402, -66.69284397573162 69.47489185423402, -66.69284397573162 19.645674168881076, -166.41728126401668 19.645674168881076))


(<function __main__.bounding_box()>,
 <shapely.geometry.polygon.Polygon at 0x7f95d94a6400>)

In [42]:
center, center_val = client_request("select ST_AsText(ST_Centroid(ST_Union(geom))) from mediumdata where state = 'California' or state = 'Texas';",0.1)

Using OS Login user [ingastrelnikova28_gmail_com] instead of requested user [ingastrelnikova28]


(-93.59382422990771, 38.76069031744346)


(<folium.folium.Map at 0x7f95aba9a280>,
 (-93.59382422990771, 38.76069031744346))

In [43]:
center, center_val = client_request("select ST_AsText(ST_Centroid(ST_Union(geom))) from mediumdata where state = 'California' or state = 'Texas';",0.1)

Using OS Login user [ingastrelnikova28_gmail_com] instead of requested user [ingastrelnikova28]


(-94.6510589585472, 39.40132193795814)


In [45]:
center